## Day 1

In [1]:
with open("input01.txt") as file:
    depths = [int(d) for d in file]

In [2]:
from itertools import zip_longest
sum([previous < current for previous,current in zip(depths, depths[1:])])

1624

In [3]:
cum_depths = [sum(window) for window in zip(depths, depths[1:], depths[2:])]
sum([previous < current for previous,current in zip(cum_depths, cum_depths[1:])])

1653

## Day 2

In [4]:
with open("input02.txt") as file:
    directions = [tuple(direction.split(" ")) for direction in file]

In [5]:
from collections import defaultdict
d = defaultdict(int)

for direction, distance in directions:
    d[direction] += int(distance)
    
vertical = d["down"] - d["up"]
vertical * d["forward"]

1524750

In [6]:
aim = 0
horizontal = 0
depth = 0

for direction, distance in directions:
    distance = int(distance)
    
    if direction == "up": aim -= distance
    if direction == "down": aim += distance
    if direction == "forward":
        horizontal += distance
        depth += aim * distance
        
horizontal * depth

1592426537

## Day 3

In [7]:
from collections import Counter

with open("input03.txt") as file:
    numbers = file.readlines()
    
matrix = list(zip(*[[n for n in number.replace("\n", "")] for number in numbers]))

counters = [Counter(digits) for digits in matrix]
gamma_rate = int("".join([counter.most_common()[0][0] for counter in counters]), base=2)
epsilon_rate = int("".join([counter.most_common()[-1][0] for counter in counters]), base=2)
gamma_rate * epsilon_rate

1307354

In [8]:
from collections import Counter
from dataclasses import dataclass

@dataclass(frozen=True)
class Metric:
    index: int
    tie_breaker: str
    
oxygen = Metric(0, "1")
co2 = Metric(1, "0")


with open("input03.txt") as file:
    numbers = [line.replace("\n", "") for line in file]
    
def find_rating(numbers, metric: Metric, current_bit=0):
    if len(numbers) == 1:
        
        return int(numbers[0], base=2)
    
    matrix = list(zip(*[[n for n in number] for number in numbers]))
    sorted_counts = Counter(matrix[current_bit]).most_common()
    most_common_digit, count = sorted_counts[metric.index]
        
    if sorted_counts[abs(metric.index-1)][1] == count:
        most_common_digit = metric.tie_breaker
        
    return find_rating(
        list(filter(lambda n: n[current_bit]==most_common_digit, numbers)),
        metric,
        current_bit+1,
    )
    
find_rating(numbers, oxygen) * find_rating(numbers, co2)

482500

## Day 4

In [9]:
import numpy as np
from numpy import ma

with open("input04.txt") as file:
    data = [line.replace("\n", "") for line in file]
    
bingo_boards = [data[2:][i*6:i*6+5] for i in range(int((len(data)-1)/6))]
bingo_boards = [[list(filter(lambda x: x != "", b.split(" "))) for b in board] for board in bingo_boards]

bingo_boards = [ma.array(board, mask=np.zeros(shape=(5,5))).astype(int) for board in bingo_boards]

bingo_numbers = [int(number) for number in data[0].split(",")]

In [67]:
def play_bingo(bingo_numbers, bingo_boards) -> int:
    for number in bingo_numbers:
        for board in bingo_boards:
            board.mask |= board==number  # evaluate number

            column_bingo = np.any(np.all(board.mask, axis=0))  # check columns for bingo
            row_bingo = np.any(np.all(board.mask, axis=1))  # check rows for bingo

            if column_bingo or row_bingo:
                 return board.sum() * number

play_bingo(bingo_numbers, bingo_boards)

4008

In [80]:
from dataclasses import dataclass
from itertools import count

@dataclass
class BingoBoard:
    array: ma.array
    rank: int = None
    number: int = None

def play_bingo_last(bingo_numbers, bingo_boards) -> int:
    ranks = count(1)
    bingo_board_instances = [BingoBoard(board) for board in bingo_boards]

    for number in bingo_numbers:
        for board in bingo_board_instances:
            if not board.rank:
                board.array.mask |= board.array==number  # evaluate number

                column_bingo = np.any(np.all(board.array.mask, axis=0))  # check columns for bingo
                row_bingo = np.any(np.all(board.array.mask, axis=1))  # check rows for bingo

                if column_bingo or row_bingo:
                    board.rank = next(ranks)
                    board.number = number

    last_bingo = sorted(bingo_board_instances, key=lambda board: board.rank, reverse=True)[0]
    print(last_bingo)
    return last_bingo.array.sum() * last_bingo.number

# reset boards
for board in bingo_boards:
    board.mask = np.zeros(board.shape)

play_bingo_last(bingo_numbers, bingo_boards)

BingoBoard(array=masked_array(
  data=[[--, --, --, 42, --],
        [--, --, 98, --, --],
        [--, --, --, 20, --],
        [56, --, --, --, 81],
        [--, --, --, --, 61]],
  mask=[[ True,  True,  True, False,  True],
        [ True,  True, False,  True,  True],
        [ True,  True,  True, False,  True],
        [False,  True,  True,  True, False],
        [ True,  True,  True,  True, False]],
  fill_value=999999), rank=100, number=97)


34726